# Comments
### This is my analysis of the data.


In [2]:
# Setting up imports.
import pandas as pd
from pathlib import Path

# Setting up the file path.
schools_complete_file = Path("Resources","schools_complete.csv")
students_complete_file = Path("Resources","students_complete.csv")

# Using the csv reader.
schools_complete = pd.read_csv(schools_complete_file)
students_complete = pd.read_csv(students_complete_file)

In [3]:
# Merging files into one data frame.
schoolwide_df = pd.merge(schools_complete,students_complete,how="left", on=["school_name","school_name"])
schoolwide_df.sort_index(ascending=True)

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84
...,...,...,...,...,...,...,...,...,...,...,...
39165,14,Thomas High School,Charter,1635,1043130,39165,Donna Howard,F,12th,99,90
39166,14,Thomas High School,Charter,1635,1043130,39166,Dawn Bell,F,10th,95,70
39167,14,Thomas High School,Charter,1635,1043130,39167,Rebecca Tanner,F,9th,73,84
39168,14,Thomas High School,Charter,1635,1043130,39168,Desiree Kidd,F,10th,99,90


## District Summary

In [4]:
# Returning the number of unique school districts.
school_count = schoolwide_df["school_name"].nunique()
school_count

15

In [5]:
# Returning the total number of students.
student_count = len(schoolwide_df["student_name"])
student_count

39170

In [6]:
# Finding the total budget of the entire school district.
budget_df = schoolwide_df[["school_name","budget"]].drop_duplicates()
budget_df.reset_index()
total_budget = budget_df["budget"].sum()
total_budget

24649428

In [7]:
# Returning average math score for the entire district.
average_math_score = schoolwide_df["math_score"].mean()
average_math_score

78.98537145774827

In [8]:
# Returning average reading score for the entire district.
average_reading_score = schoolwide_df["reading_score"].mean()
average_reading_score

81.87784018381414

In [9]:
# Finding the percent of students that passed math.
passing_math_count = schoolwide_df[(schoolwide_df["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [10]:
# Finding the percent of students that passed reading.
passing_reading_count = schoolwide_df[(schoolwide_df["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [11]:
# Finding the percent of students that passed both math and reading.
passing_math_reading_count = schoolwide_df[(schoolwide_df["math_score"] >= 70) & (schoolwide_df["reading_score"] >= 70)].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [12]:
# Creating a high-level snapshot of the district overall.
district_summary_df = pd.DataFrame(
    {"Total Schools":[school_count], "Total Students":[student_count], "Total Budget":[total_budget], 
     "Average Math Score":[average_math_score], "Average Reading Score":[average_reading_score],
    "% Passing Math":[passing_math_percentage], "% Passing Reading":[passing_reading_percentage],
    "% Overall Passing":[overall_passing_rate]}
)

# Formatting
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [13]:
# Selecting the type per school.
school_types = schools_complete.set_index(["school_name"])["type"]
school_types.sort_index(ascending=True)

school_name
Bailey High School       District
Cabrera High School       Charter
Figueroa High School     District
Ford High School         District
Griffin High School       Charter
Hernandez High School    District
Holden High School        Charter
Huang High School        District
Johnson High School      District
Pena High School          Charter
Rodriguez High School    District
Shelton High School       Charter
Thomas High School        Charter
Wilson High School        Charter
Wright High School        Charter
Name: type, dtype: object

In [14]:
# Calculating the number of students per school type.
per_school_counts = schools_complete.set_index(["school_name"])["size"]
per_school_counts.sort_index(ascending=True)

school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: size, dtype: int64

In [15]:
# Calculating total school budget and per capita budget per school.
school_budget = schools_complete.set_index(["school_name"])["budget"]
school_per_capita = school_budget / per_school_counts
school_budget.sort_index(ascending=True)

school_name
Bailey High School       3124928
Cabrera High School      1081356
Figueroa High School     1884411
Ford High School         1763916
Griffin High School       917500
Hernandez High School    3022020
Holden High School        248087
Huang High School        1910635
Johnson High School      3094650
Pena High School          585858
Rodriguez High School    2547363
Shelton High School      1056600
Thomas High School       1043130
Wilson High School       1319574
Wright High School       1049400
Name: budget, dtype: int64

In [16]:
# Calculating average math and reading scores per school.
per_school_math = students_complete.groupby(["school_name"])["math_score"].mean()
per_school_reading = students_complete.groupby(["school_name"])["reading_score"].mean()
per_school_reading

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [17]:
# Listing students that are passing math
students_passing_math = schoolwide_df[schoolwide_df["math_score"] >= 70]
school_student_math = students_passing_math.groupby("school_name").size()
school_student_math.sort_index(ascending=True)

school_name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
dtype: int64

In [18]:
# Listing students that are passing reading
students_passing_reading = schoolwide_df[schoolwide_df["reading_score"] >= 70]
school_student_reading = students_passing_reading.groupby("school_name").size()
school_student_reading.sort_index(ascending=True)

school_name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
dtype: int64

In [19]:
# Using the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = schoolwide_df[
    (schoolwide_df["reading_score"] >= 70) & (schoolwide_df["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()
school_students_passing_math_and_reading.sort_index(ascending=True)

school_name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
dtype: int64

In [20]:
# Using the provided code to calculate the passing rates
per_school_passing_math = school_student_math / per_school_counts * 100
per_school_passing_reading = school_student_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100
per_school_passing_math.sort_index(ascending=True)

school_name
Bailey High School       66.680064
Cabrera High School      94.133477
Figueroa High School     65.988471
Ford High School         68.309602
Griffin High School      93.392371
Hernandez High School    66.752967
Holden High School       92.505855
Huang High School        65.683922
Johnson High School      66.057551
Pena High School         94.594595
Rodriguez High School    66.366592
Shelton High School      93.867121
Thomas High School       93.272171
Wilson High School       93.867718
Wright High School       93.333333
dtype: float64

In [21]:
# Creating a full summary of each school using the variables created.
per_school_summary = pd.DataFrame(
    {"School Type":school_types, "Total Students":per_school_counts, "Total School Budget":school_budget,
    "Per Student Budget":school_per_capita, "Average Math Score":per_school_math, "Average Reading Score":per_school_reading,
    "% Passing Math":per_school_passing_math, "% Passing reading":per_school_passing_reading, "% Overall Passing":overall_passing_rate}
)

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Highest-Performing Schools (by % Overall Passing)

In [22]:
# Sorting the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [23]:
# Sorting the schools by `% Overall Passing` in ascending order and display the bottom 5 rows.
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=True)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [34]:
students_by_grade = students_complete.groupby("grade")
students_by_grade[["math_score"]].mean()

,math_score
grade,
10th,78.941483
11th,79.083548
12th,78.993164
9th,78.935659


## Reading Score by Grade 

In [39]:
students_by_grade[["reading_score"]].mean()

,reading_score
grade,
10th,81.874410
11th,81.885714
12th,81.819851
9th,81.914358


## Scores by School Spending

In [63]:
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
school_spending_bins = school_budget / per_school_counts
school_spending_df = pd.cut(school_spending_bins, spending_bins, labels = labels, ordered = True)
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()
spending_summary.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": overall_passing_rate
})


## Scores by School Size

## Scores by School Type